In [ ]:
import onnx
from onnx import helper, TensorProto
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.core.datatype import DataType
import numpy as np
from qonnx.util.basic import gen_finn_dt_tensor
import onnx.version_converter as vc
from onnx.backend.test.case.node import expect

from qonnx.transformation.infer_shapes import InferShapes


In [1]:
class slice_template():
    def __init__(self):
        self.inp_shp = []
        self.out_shp = []
        self.slice_shape = []
        self.idt = DataType["UINT8"]
        self.param_dt = DataType["INT64"]
        self.input = helper.make_empty_tensor_value_info()
        self.output = helper.make_empty_tensor_value_info()
        self.slice_attr = {}
        self.opset_version = helper.make_operatorsetid("", 9)

# input_shape = [4]
# output_shape = [2]
# param_shape = [1]
# idt = DataType["UINT8"]
# param_dt = DataType["INT64"]
# slice_in = helper.make_tensor_value_info("735", TensorProto.INT64, input_shape)
# slice_out = helper.make_tensor_value_info("slice_out", TensorProto.INT64, output_shape)
# slice_attr = {}
# slice_attr["starts"] = np.array([0],dtype=np.int64)
# slice_attr["ends"] = np.array([2],dtype=np.int64)
# slice_attr["axes"] = np.array([0],dtype=np.int64)

    def make_node(self):
        self.slice_node = helper.make_node(
            "Slice",
            # Need to set to the name of the tensor
            inputs=[self.input],
            # Need to set to the name of the tensor
            outputs=[self.output],
            **self.slice_attr
        )

    def make_model(self):

        model_config = {}
        model_config["opset_imports"] = [self.opset_version]

        self.model = ModelWrapper(helper.make_model(
            helper.make_graph(
                [self.slice_node],
                inputs=[self.input],
                outputs=[self.output],
                name="slice_graph"
            ), **model_config)
        )
        self.model.transform(InferShapes())
        onnx.checker.check_model(self.model.model)
        self.model.save("onnx_model/Slice_model.onnx")


NameError: name 'DataType' is not defined